# Import the Zenbase Library

In [ ]:
import sys
import subprocess

def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    except subprocess.CalledProcessError as e:
        print(f"Failed to install {package}: {e}")
        raise

def install_packages(packages):
    for package in packages:
        install_package(package)

try:
    # Check if running in Google Colab
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    # Install the zenbase package if running in Google Colab
    # install_package('zenbase')
    # Install the zenbse package from a GitHub branch if running in Google Colab
    install_package('git+https://github.com/zenbase-ai/lib.git@main#egg=zenbase&subdirectory=py')

    # List of other packages to install in Google Colab
    additional_packages = [
        'python-dotenv',
        'langfuse',
        'openai',
        'langchain',
        'langchain_openai'
    ]
    
    # Install additional packages
    install_packages(additional_packages)

# Now import the zenbase library
try:
    import zenbase
except ImportError as e:
    print("Failed to import zenbase: ", e)
    raise

# Configure the Environment

In [4]:
from pathlib import Path
from dotenv import load_dotenv

# import os
#
# os.environ["OPENAI_API_KEY"] = "..."
# os.environ["LANGFUSE_HOST"] = "..."
# os.environ["LANGFUSE_PUBLIC_KEY"] = "..."
# os.environ["LANGFUSE_SECRET_KEY"] = "..."

load_dotenv(Path("../../.env.test"), override=True)

True

In [5]:
import nest_asyncio

nest_asyncio.apply()

# Initial Setup

In [6]:
from openai import OpenAI
from langfuse import Langfuse

langfuse = Langfuse()
langfuse.auth_check()

openai = OpenAI()

# What you already have should look like below:

## Your OpenAI Call should look like this with LangChain (It could be with OpenAI too, doesn't matter)

In [7]:
from langfuse.decorators import observe

@observe()
def langchain_chain(inputs):
    from langchain_openai import ChatOpenAI
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_core.output_parsers import StrOutputParser

    messages = [
        (
            "system",
            "You are an expert math solver. Your answer must be just the number with no separators, and nothing else. Follow the format of the examples.",
        ),
        ("user", "{question}")
    ]

    chain = (
        ChatPromptTemplate.from_messages(messages)
        | ChatOpenAI(model="gpt-3.5-turbo")
        | StrOutputParser()
    )

    answer = chain.invoke(inputs)
    return answer


## Your Scoring Function should look like this:

In [ ]:
def score_answer(answer: str, expected_output: dict):
    """The first argument is the return value from the `langchain_chain` function above."""
    score = int(answer == expected_output.split("#### ")[-1])
    langfuse.score(
        name="correctness",
        value=score,
        trace_id=langfuse.get_trace_id(),
    )
    return {"score": score}

## Your Evaluation should look like this:

In [8]:
evalset = langfuse.get_dataset("gsm8k-testset")

scores = []
for item in evalset.items:
   answer = langchain_chain(item.input)
   eval = score_answer(answer, item.expected_output)
   scores.append(eval["score"])

print("Average score", sum(scores) / len(scores))

Average score 0.4


# How you should do the few-shot learning

## Rewrite your langchain_chain function to use the `zenbase` decorators

In [18]:
from zenbase.types import LMRequest, LMDemo, deflm

@deflm
@observe()
def zen_chain(request: LMRequest):
    from langchain_openai import ChatOpenAI
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_core.output_parsers import StrOutputParser

    messages = [
        (
            "system",
            "You are an expert math solver. Your answer must be just the number with no separators, and nothing else. Follow the format of the examples.",
        )
    ]

    for demo in request.zenbase.task_demos:
        messages += [
            ("user", demo.inputs["question"]),
            ("assistant", demo.outputs),
        ]

    messages.append(("user", "{question}"))

    chain = (
        ChatPromptTemplate.from_messages(messages)
        | ChatOpenAI(model="gpt-3.5-turbo")
        | StrOutputParser()
    )

    answer = chain.invoke(request.inputs)
    return answer


## Make your evaluation function to use the langfuse from the LMDemo and the langfuse from the Zenbase

In [ ]:
def score_answer(answer: str, demo: LMDemo, langfuse: Langfuse):
    """The first argument is the return value from the `zen_chain` function above."""
    score = int(answer == demo.outputs.split("#### ")[-1])
    langfuse.score(
        name="correctness",
        value=score,
        trace_id=langfuse.get_trace_id(),
    )
    return {"score": score}

## Optimize the few-shot learning

### Define your optimizer:


In [19]:
from zenbase.optim.metric.labeled_few_shot import LabeledFewShot
from zenbase.adaptors.langfuse_helper import ZenLangfuse

optimizer = LabeledFewShot(
    demoset=ZenLangfuse.dataset_demos(langfuse.get_dataset("gsm8k-testset")), ## The dataset to use for the few-shot learning and training
    shots=3,
)

60 If each chicken eats 3 cups of feed per day, then for 20 chickens they would need 3*20=<<3*20=60>>60 cups of feed per day.
If she feeds the flock 15 cups of feed in the morning, and 25 cups in the afternoon, then the final meal would require 60-15-25=<<60-15-25=20>>20 cups of chicken feed.
#### 20
James runs 3 sprints per session, so he runs 3 * 60 = <<3*60=180>>180 meters per session.
He runs 3 sessions per week, so he runs 180 * 3 = <<180*3=540>>540 meters per week.
#### 540 He sprints 3*3=<<3*3=9>>9 times
So he runs 9*60=<<9*60=540>>540 meters
#### 540
To calculate the profit made by Josh, we need to find the total cost (purchase price + repairs) of buying and fixing the house, which is $80,000 + $50,000 = $130,000.

Since the value of the house increased by 150%, the new value of the house is $80,000 + $80,000 * 1.5 = $80,000 + $120,000 = $200,000.

Therefore, the profit made by Josh is $200,000 - $130,000 = $70,000. 

So, the profit Josh made is $70,000. The cost of the house a

### Perform the optimization

In [ ]:
best_fn, candidate_results, _ = optimizer.perform(
    zen_chain,
    evaluator=ZenLangfuse.metric_evaluator(
        evalset=evalset,
        evaluate=score_answer,
        langfuse=langfuse,
    ),
    samples=4,
    concurrency=1,
    rounds=1,
)

### Use the best function

In [20]:
output = best_fn({"question": "What is 2+2?"})
output

'4'

### Save the best function

In [21]:
# You can also save the zenbase params for re-use
import pickle

pickled_zenbase = pickle.dumps(best_fn.zenbase)
zen_chain.zenbase = pickle.loads(pickled_zenbase)

zen_chain({"question": "What is 2 + 2?"}) # uses the best few-shot demos

'4'